In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re
import spacy



import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Input, GRU, Bidirectional, GlobalMaxPooling1D,GlobalAveragePooling1D, Dropout,SpatialDropout1D,Conv1D,concatenate,Flatten, SimpleRNN, LSTM
#Bidirectional은 양방향으로 데이터를 보기 위해서 사용되는 함수
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

2022-10-29 00:39:09.036606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-10-29 00:39:09.036705: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-10-29 00:39:09.036763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-10-29 00:39:09.036817: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusparse.so.11'; dlerror: libcusparse.so.11: cannot open shared object file: No such file or directory
2022-10-29 00:39:09.036854: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Canno

In [5]:
#데이터 불러오기
data = pd.read_csv('/home/hsung/ds_study/Vit/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
cutoff = int(len(data)*0.7)
train_data = data[:cutoff]
test_data = data[cutoff:]

# stopwords

In [9]:
#!python -m spacy download en_core_web_sm

In [10]:
nlp = spacy.load('en_core_web_sm')

In [11]:
# loading stop words
stop_words = nlp.Defaults.stop_words
# removing 'not' word from stop words list because this word has enough effect on sentiment
nlp.Defaults.stop_words.remove('not')
nlp.vocab['not'].is_stop = False

In [14]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    
    # remove stop words
    raw = []
    for word in text.split():
        if word in stop_words:
            pass
        else:
            raw.append(word)
    text = ' '.join(raw)
    # lemmatization
    text = ' '.join([i.lemma_ for i in nlp(text)])

    return text

In [15]:
X_train_raw = train_data['review'].map(lambda com : clean_text(com))
X_test_raw = test_data['review'].map(lambda com : clean_text(com))

Y_train = train_data['sentiment']

In [16]:
print("X train shape : ", X_train_raw.shape)
print("X test shape : ", X_test_raw.shape)
print("y train shape : ", Y_train.shape)

X train shape :  (35000,)
X test shape :  (15000,)
y train shape :  (35000,)


In [17]:
max_features=90000
maxlen=90
embed_size=300


tok=Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(X_train_raw)
tok.fit_on_texts(X_test_raw)
X_train = sequence.pad_sequences(tok.texts_to_sequences(X_train_raw),
                        maxlen = maxlen, truncating = "pre")
X_test = sequence.pad_sequences(tok.texts_to_sequences(X_test_raw),
                       maxlen = maxlen, truncating = "pre")

In [18]:
word_index = tok.word_index

In [21]:
word_index = {e:i for e,i in word_index.items() if i <= max_features}

In [22]:
x_train, x_val, y_train, y_val = train_test_split(X_train,Y_train,train_size=0.9, random_state=199)
print(x_train.shape," ",y_train.shape," ",x_val.shape," ",y_val.shape)

(31500, 90)   (31500,)   (3500, 90)   (3500,)


# Word2vec

In [38]:
import gensim


word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/home/hsung/ds_study/Vit/New/GoogleNews-vectors-negative300.bin.gz', binary=True)

print('모델의 크기(shape) :',word2vec_model.vectors.shape) # 모델의 크기 확인


모델의 크기(shape) : (3000000, 300)


In [39]:
embedding_matrix = np.zeros((max_features, 300))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (90000, 300)


In [40]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [44]:
for word, index in word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = get_vector(word)
    if vector_value is not None:
        embedding_matrix[index-1] = vector_value

# 모델 구성하기

In [45]:
input = Input(shape=(maxlen, )) 
layer = Embedding(max_features, 300, weights=[embedding_matrix], input_length=maxlen, trainable=True)(input)
layer = SimpleRNN(128)(layer)   
layer = Dense(50,activation='relu')(layer)
layer = Flatten()(layer)
layer= Dense(2, activation="sigmoid")(layer)
model = Model(inputs = input, outputs = layer)  
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

2022-10-29 02:00:36.910401: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90)]              0         
                                                                 
 embedding (Embedding)       (None, 90, 300)           27000000  
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               54912     
                                                                 
 dense (Dense)               (None, 50)                6450      
                                                                 
 flatten (Flatten)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 102       
                                                                 
Total params: 27,061,464
Trainable params: 27,061,464
Non-tra

In [46]:
file_path = '{epoch:02d}-{val_loss:.5f}.h5'
checkpoint = ModelCheckpoint(file_path,monitor='val_loss',verbose = 1, save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss',patience=1)

In [47]:
y_train[y_train=='positive'] = 1
y_train[y_train=='negative'] = 0
y_val[y_val=='positive'] = 1
y_val[y_val=='negative'] = 0

In [48]:
from tensorflow.keras.utils import to_categorical 
y_train_one = to_categorical(y_train)
y_val_one = to_categorical(y_val)

In [49]:
hist = model.fit(x_train, y_train_one, epochs=3,batch_size=256, validation_data=(x_val,y_val_one), callbacks= [checkpoint,early_stop])

Epoch 1/3
124/124 [==============================] - ETA: 0s - loss: 0.5598 - accuracy: 0.6886
Epoch 1: val_loss improved from inf to 0.42581, saving model to 01-0.42581.h5
124/124 [==============================] - 31s 247ms/step - loss: 0.5598 - accuracy: 0.6886 - val_loss: 0.4258 - val_accuracy: 0.8140
Epoch 2/3
124/124 [==============================] - ETA: 0s - loss: 0.2744 - accuracy: 0.8902
Epoch 2: val_loss improved from 0.42581 to 0.39563, saving model to 02-0.39563.h5
124/124 [==============================] - 29s 233ms/step - loss: 0.2744 - accuracy: 0.8902 - val_loss: 0.3956 - val_accuracy: 0.8200
Epoch 3/3
124/124 [==============================] - ETA: 0s - loss: 0.1424 - accuracy: 0.9499
Epoch 3: val_loss did not improve from 0.39563
124/124 [==============================] - 29s 233ms/step - loss: 0.1424 - accuracy: 0.9499 - val_loss: 0.4227 - val_accuracy: 0.8380


In [55]:
import matplotlib.pyplot as plt
def acc_loss_plot(hist):
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc = 'upper left')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='upper right')

    plt.show()

In [56]:
y_test = test_data['sentiment'] 
y_test[y_test=='positive'] = 1
y_test[y_test=='negative'] = 0
y_test_one = to_categorical(y_test)

In [57]:
y_test_one

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [58]:
score = model.evaluate(X_test,y_test_one)

469/469 [==============================] - 6s 13ms/step - loss: 0.4260 - accuracy: 0.8430


In [59]:
score[1]

0.8429999947547913